In [198]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, normalize

In [199]:
af = pd.read_csv("/Users/thomascurran/Desktop/MACS Final Project v2/AF_Data.csv")
descriptors = pd.read_csv("/Users/thomascurran/Desktop/MACS Final Project v2/Descriptors_Census.csv")
enrollment = pd.read_csv("/Users/thomascurran/Desktop/MACS Final Project v2/Enrollment.csv")
test_scores = pd.read_csv("/Users/thomascurran/Desktop/MACS Final Project v2/school_test_scores(14-16).csv")

In [200]:
school_types = enrollment[['Year','SchoolCode','SchoolType']]
df = af[['Year','SchoolCode','SchoolName','IndexScore']]

In [201]:
df2 = pd.merge(df, 
              descriptors, 
              on = ["SchoolCode", "Year"], 
              how = "inner").drop(["Site","DistrictCode"],axis = 1)
df2 = pd.merge(df2, school_types, on = ["SchoolCode","Year"], how = "inner")
df2.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/nn_df_descriptors.csv")

In [202]:
df3 = pd.merge(
    df2,
    enrollment,
    on = ["SchoolCode", "Year"],
    how = "inner").drop(["SchoolName_y","SchoolType_y"], axis = 1).rename(columns = {"SchoolName_x":"SchoolName","SchoolType_x": "SchoolType"})

df3.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/nn_df_descriptors&enrollment.csv")

In [203]:
df4 = pd.merge(
    df3,
    test_scores,
    how = "inner",
    on = ["SchoolCode","Year"]
).drop(["Site"], axis = 1)

df4.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/nn_df_descriptors&enrollment&testscores.csv")

In [204]:
hs = df4[df4.SchoolType == "High School"]
hs = hs.drop("SchoolType", axis = 1)
es = df4[df4.SchoolType == 'Elementary School']
es = es.drop("SchoolType", axis = 1)
ms = df4[df4.SchoolType == 'Middle School']
ms = ms.drop("SchoolType", axis = 1)

In [205]:
def cols_to_drop(df):
    
    #array of columns to drop
    dropped_cols = []
    
    #check to see if column is completely null if the number of nulls is greater than or equal to the length of the index
    for i, c in enumerate(df):
        if((df[c].isna().sum()/len(df.index)) > .50):
            dropped_cols.append(c)
         
    #drop the columns from the data frame
    df = df.drop(dropped_cols, axis = 1)
    return(df)

In [206]:
es = cols_to_drop(es)
es.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/es.csv")

ms = cols_to_drop(ms)
ms.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/ms.csv")

hs = cols_to_drop(hs)
hs.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/hs.csv")

In [231]:

#len(ms.columns)

,Year,SchoolCode,SchoolName,IndexScore,AvgIncome,AvgPropValADM,Unemployment,PovertyRate,SingleParentFam,FRL,...,8_Math_Satisfactory,8_Reading_Advanced,8_Reading_Below_Satisfactory,8_Reading_Satisfactory,8_Science_Advanced,8_Science_Below_Satisfactory,8_Science_Satisfactory,EOI_Algebra_Advanced,EOI_Algebra_Below_Satisfactory,EOI_Algebra_Satisfactory
622,2016,09I034510,LESLIE F ROBLYER LEARNING CTR,73.0,60315.50,24895.16,0.07,0.14,0.37,0.790,...,0.498473,0.149484,0.166129,0.684388,0.115825,0.403876,0.480299,0.542743,0.054739,0.402518
696,2016,16I001530,CACHE 5TH AND 6TH GRADE MS,85.0,93637.81,61263.07,0.07,0.11,0.26,0.495,...,0.498473,0.149484,0.166129,0.684388,0.115825,0.403876,0.480299,0.542743,0.054739,0.402518
1032,2016,72I011515,OWASSO 6TH GRADE CTR,87.0,84896.09,52132.65,0.05,0.06,0.21,0.324,...,0.498473,0.149484,0.166129,0.684388,0.115825,0.403876,0.480299,0.542743,0.054739,0.402518
1231,2016,08I167505,FORT COBB-BROXTON MS,74.0,51494.01,25787.81,0.07,0.17,0.31,0.712,...,0.465000,0.193200,0.080000,0.726800,0.107100,0.370000,0.522900,0.360000,0.100000,0.540000
1234,2016,09I022510,PIEDMONT MS,91.0,96974.43,43118.98,0.03,0.04,0.16,0.205,...,0.356000,0.332500,0.050000,0.617500,0.400200,0.130000,0.469800,1.000000,0.000000,0.000000


In [225]:
imputer = Imputer()
ms.iloc[:,3:58] = imputer.fit_transform(ms.iloc[:,3:58])
hs.iloc[:, 3:52] = imputer.fit_transform(hs.iloc[:,3:52])
es.iloc[:, 3:55] = imputer.fit_transform(es.iloc[:,3:55])
ms.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/ms_clean.csv")
es.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/es_clean.csv")
hs.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/hs_clean.csv")

In [239]:
ms_test = ms.copy()
#hs_test = hs_test.drop(["SchoolName","SchoolCode","Year"], axis = 1)
ms_test.iloc[:,3:len(ms.columns)] = normalize(ms_test.iloc[:,3:len(ms.columns)])

In [241]:
ms_test.to_csv("/Users/thomascurran/Desktop/MACS Final Project v2/ms_test_normalize.csv")